In [1]:
from grafo import Grafo
import pandas as pd

In [7]:
cruces = pd.read_csv("data/cruces_clean.csv")
calles = pd.read_csv("data/direcciones_clean.csv")

In [3]:
cruces

,id_via,nombre_via,id_via_cruzada,nombre_via_cruzada,x,y
0,127,CALLE DE ISABEL COLBRAND,2792,CALLE DE CASTIELLO DE JACA,44260746,448479753
1,127,CALLE DE ISABEL COLBRAND,1837,CALLE DE FEDERICO MOMPOU,44260732,448479748
2,127,CALLE DE ISABEL COLBRAND,87100,CALLE DE MARÍA TUBAU,44290433,448494484
3,127,CALLE DE ISABEL COLBRAND,2793,PASEO DE TIERRA DE MELIDE,44320997,448510385
4,127,CALLE DE ISABEL COLBRAND,2794,CALLE DE PORTOMARÍN,44322285,448530840
...,...,...,...,...,...,...
36160,31003706,CALLE DE TREN OBRERO,31003703,CALLE DE JOSÉ MARÍA LOZANO SAINZ,44083665,446665463
36161,31004081,GLORIETA DE CIRILO MARTÍN MARTÍN,369600,CARRETERA DE HÚMERA,43390886,447883842
36162,31005685,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ,29900,CALLE DE ALICANTE,44145890,447199432
36163,31005685,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ,2729,CALLE DEL GRANITO,44150370,447189024


In [15]:
g = Grafo()
for id_via in cruces["id_via"].drop_duplicates():
    g.agregar_vertice(id_via)



0             127
6             150
10            200
14            300
17            400
           ...   
36158    31003598
36159    31003703
36160    31003706
36161    31004081
36162    31005685
Name: id_via, Length: 7224, dtype: int64

In [8]:
calles

,Direccion completa,Codigo de via,x,y
0,AUTOV A-1 1000 EN,31001337,44305633,448250340
1,AUTOV A-1 1000 SA,31001337,44312246,448249077
2,AUTOV A-1 1100 EN,31001337,44367522,448330933
3,AUTOV A-1 1100 SA,31001337,44373755,448325151
4,AUTOV A-1 1200 EN,31001337,44425207,448408185
...,...,...,...,...
210352,PASEO ZURRÓN 49,725,45051964,448025888
210353,PASEO ZURRÓN 51,725,45058080,448025960
210354,PASEO ZURRÓN 53,725,45061025,448027575
210355,PASEO ZURRÓN 55,725,45063500,448027080
